In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import xarray as xr
import altair as alt

alt.data_transformers.disable_max_rows()

import pandas as pd

from google.cloud import storage

In [ ]:
def _(df, *args, **kwargs):
    print(len(df))
    display(df.head(*args, **kwargs))

In [ ]:
import h5netcdf

In [ ]:
df = xr.open_dataset(
    f"gcs://solar-pv-nowcasting-data/PV/uk_pv/trimmed_v1/5min.netcdf", engine="h5netcdf"
)

In [ ]:
storage_client = storage.Client()

In [ ]:
file = "solar-pv-nowcasting-data/NWP/UK_Met_Office/UKV_intermediate_version_3.zarr"

In [ ]:
nwp = xr.open_dataset(
    f"gcs://{file}",
    engine="zarr",
    consolidated=True,
    mode="r",
    chunks="auto",
)

In [ ]:
nwp

In [ ]:
nwp.coords["x"].size

In [ ]:
# n_x = nwp.coords['x'].size
# n_y = nwp.coords['y'].size
x = (
    nwp.isel(
        init_time=0,
        step=slice(0, 37, 10),
        variable=10,
        x=slice(0, 100),
        y=slice(0, 100),
    )
    #     .sel(x=slice(None, None, ),
    #          y=slice(None, None, nwp.coords['y'].size // 100))
    ["UKV"]
)
x

In [ ]:
for step in range(x.coords["step"].size):
    data = (
        x.isel(step=step).to_pandas().stack().reset_index().rename(columns={0: "value"})
    )
    _(data)
    display(
        alt.Chart(data)
        .mark_rect()
        .encode(x="x:O", y="y:O", color="value:Q")
        .properties(width=300, height=300)
    )